In [31]:
from collections import defaultdict
from multiprocessing import Pool, TimeoutError

In [24]:
def gatherStats(file):    
    infoMap = defaultdict()
    with open(file, 'r') as f:
        while (f.readline()[0] == '@'):
            continue
        for l in f:
            cols = l.split('\t')
            readParts = cols[0].split(';')
            rid = readParts[0].split('/')[0].split('read')[1]
            tid = readParts[0].split('/')[1]
            mate1pos = readParts[1].split(':')[1].split('-')[0]
            mate2pos = readParts[2].split(':')[1].split('-')[0]

            aligned_tid = cols[3]
            isPrimary1 = (cols[1] == '99')
            isPrimary2 = (cols[1] == '147')
            aligned_rpos = cols[3]
            isExact = (mate1pos == aligned_rpos or mate2pos == str(int(aligned_rpos)+1))
            if rid not in infoMap:
                infoMap[rid] = [0,0,0,0]
            infoMap[rid][0]+=1
            if (isPrimary1):
                infoMap[rid][1]+=1
            elif (isPrimary2):
                infoMap[rid][2]+=1
            if isExact:
                infoMap[rid][3]+=1
    return infoMap

In [34]:
def calcLineStat(lines):
    """Make a dict out of the parsed, supplied lines"""
    infoMap = defaultdict()
    for l in lines.split('\n'):
        cols = l.split('\t')
        readParts = cols[0].split(';')
        rid = readParts[0].split('/')[0].split('read')[1]
        tid = readParts[0].split('/')[1]
        mate1pos = readParts[1].split(':')[1].split('-')[0]
        mate2pos = readParts[2].split(':')[1].split('-')[0]

        aligned_tid = cols[3]
        isPrimary1 = (cols[1] == '99')
        isPrimary2 = (cols[1] == '147')
        aligned_rpos = cols[3]
        isExact = (mate1pos == aligned_rpos or mate2pos == str(int(aligned_rpos)+1))
        if rid not in infoMap:
            infoMap[rid] = [0,0,0,0] # total count, mate1 is primary, mate2 is primary, correct position
        infoMap[rid][0]+=1
        if (isPrimary1):
            infoMap[rid][1]+=1
        elif (isPrimary2):
            infoMap[rid][2]+=1
        if isExact:
            infoMap[rid][3]+=1
    return infoMap

def para_gatherStats(file, numthreads=9):
    numlines = 100

    lines = open(file).readlines()

    # create the process pool
    pool = Pool(processes=numthreads)

    # map the list of lines into a list of result dicts
    result_list = pool.map(calcLineStat, 
        (lines[line:line+numlines] for line in range(0,len(lines),numlines) ) )

    # reduce the result dicts into a single dict
    result = {}
    map(result.update, result_list)
    return result

In [ ]:
puffMapPara=para_gatherStats('/mnt/scratch2/puff_out.sam')

In [26]:
kalMap = gatherStats('/mnt/scratch2/kal_out.sam')
rapMap = gatherStats('/mnt/scratch2/rapmap_out.sam')
puffMap = gatherStats('/mnt/scratch2/puff_out.sam')

In [27]:
print('actual # of reads: 33,464,798')
print(len(puffMap))
print(len(rapMap))
print(len(kalMap))

actual # of reads: 33,464,798
33431705
33462849
33464798


In [15]:
good_cnt = 0
bad_cnt = 0
for k,v in readMap.items():
    if (v[0] > 10):
        bad_cnt+=1
        #print('{}:  {},{},{},{}'.format(k, v[0], v[1], v[2], v[3]))
    else:
        good_cnt+=1
print(good_cnt)
print(bad_cnt)

33213471
218234


In [ ]:
def f(x):
    return x*x

if __name__ == '__main__':
    with Pool(5) as p:
        print(p.map(f, [1, 2, 3]))